## Imports

In [1]:
import os
import pickle
import numpy as np
from tqdm import tqdm

# Qiskit and Physics Imports
from qiskit_aer.noise import NoiseModel
from qiskit.circuit.library import TwoLocal

# Custom Project Imports
from src.clp_zne.qpu_info.layout_cycles import heron_layout_cycles_12q
from src.clp_zne.hamiltonians import sherrington_kirkpatrick_model
from src.clp_zne.qpu_info.backends import FakeTorino
from src.clp_zne.mitigate import clp_zne_mitigate_general_topology_circuit
from src.clp_zne.backends import NClusterBackend
from src.clp_zne.utils import compute_evals_ideal, get_grid_entanglement

## Global Configuration & Parameters

In [6]:
# Simulation Settings
ROWS, COLS = 3, 3
NUM_QUBITS = ROWS * COLS
NUM_CIRCUITS = 20
NUM_OBSERVABLES = 100
SEED = 42

# Paths
OUTPUT_FOLDER = r"data\SK_histogram_square_topology_full_noise"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

## Backend & Noise Model Setup

In [3]:
# Hardware Setup
source_backend = FakeTorino()
# For square grid 3 layouts is enough
selected_layouts = [heron_layout_cycles_12q[i] for i in [5, 9, 13]]

backend = NClusterBackend.from_backend(
    cluster_size=NUM_QUBITS, 
    source_backend=source_backend,
    list_of_index_groups=selected_layouts, 
    seed=SEED
)
# The backend has 3 clusters
# Each cluster has NUM_QUBITS all-to-all connected qubits
# No connections present inbetween the clusters

# Noise Model Configuration
noise_model = NoiseModel.from_backend(
    backend, 
    readout_error=False, 
    gate_error=True, 
    thermal_relaxation=True
)

## Circuit & Observable Generation

In [4]:
# 1. Define Entanglement and Ansatz
entanglement_map = get_grid_entanglement(ROWS, COLS)
ansatz = TwoLocal(NUM_QUBITS, ['rx', 'rz'], 'cz', entanglement=entanglement_map, reps=3)

# 2. Generate Randomized Circuits
circuits = []
for i in range(NUM_CIRCUITS):
    rng = np.random.default_rng(i)
    circuit = ansatz.copy()
    parameters = rng.uniform(-np.pi, np.pi, circuit.num_parameters)
    circuit.assign_parameters(parameters, inplace=True)
    circuits.append(circuit)

# 3. Define Observables
observables = [
    sherrington_kirkpatrick_model(NUM_QUBITS, h=1, seed=i) 
    for i in range(NUM_OBSERVABLES)
]

## Execution

In [5]:
all_evals_ideal = []
all_evals_mitigated = []
all_evals_noisy = []
all_error_sums = []

# Choose layouts for cyclic permutations
layouts_for_cyclic_permutations = np.arange(backend.num_qubits).reshape((backend.num_clusters, -1)).tolist()

for circ in tqdm(circuits, desc="Processing Circuits"):
    # Ideal Evaluation
    evals_ideal = compute_evals_ideal(circ, observables)
    
    # Mitigated Evaluation
    evals_mitigated, evals_noisy, error_sums = clp_zne_mitigate_general_topology_circuit(
        circ, 
        observables,
        layouts_for_cyclic_permutations, 
        backend, 
        noise_model
    )
    
    # Store results
    all_evals_ideal.append(evals_ideal)
    all_evals_mitigated.append(evals_mitigated)
    all_evals_noisy.append(evals_noisy)
    all_error_sums.append(error_sums)

Processing Circuits:   0%|                                                                      | 0/20 [00:00<?, ?it/s]

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations



  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

 39%|███████████████████████████████▏                                                | 39/100 [00:00<00:00, 382.31it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 393.71it/s]

  4%|███                                                                                | 1/27 [00:00<00:12,  2.14it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 531.90it/s]

  7%|██████▏                                                                            | 2/27 [00:00<00:10,  2.40it/s]

  0%|                          

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations



  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 847.44it/s]

  4%|███                                                                                | 1/27 [00:00<00:06,  4.25it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 729.93it/s]

  7%|██████▏                                                                            | 2/27 [00:00<00:06,  4.16it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|██████████████████████████

...


  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 709.18it/s]

  4%|███                                                                                | 1/27 [00:00<00:06,  3.77it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 786.93it/s]

  7%|██████▏                                                                            | 2/27 [00:00<00:06,  4.05it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|██████████████████████████

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations



  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 764.02it/s]

  4%|███                                                                                | 1/27 [00:00<00:06,  3.90it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 714.27it/s]

  7%|██████▏                                                                            | 2/27 [00:00<00:06,  3.89it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|██████████████████████████

[[0.0727674  0.         0.07834151 0.        ]
 [0.20812224 0.         0.16660966 0.        ]
 [0.08860162 0.         0.11011029 0.        ]]
Running density matrix simulations



  0%|                                                                                           | 0/27 [00:00<?, ?it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 682.96it/s]

  4%|███                                                                                | 1/27 [00:00<00:07,  3.70it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 724.22it/s]

  7%|██████▏                                                                            | 2/27 [00:00<00:06,  3.89it/s]

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

100%|██████████████████████████

## Data Export

In [7]:
# Map of filenames to data
data_to_save = {
    'evals_ideal.npy': np.array(all_evals_ideal),
    'evals_mitigated.npy': np.array(all_evals_mitigated),
    'evals_noisy.npy': np.array(all_evals_noisy),
    'error_sums.npy': np.array(all_error_sums)
}

for filename, data in data_to_save.items():
    path = os.path.join(OUTPUT_FOLDER, filename)
    np.save(path, data)

print(f"Successfully saved {len(data_to_save)} files to: {OUTPUT_FOLDER}")

Successfully saved 4 files to: data\SK_histogram_square_topology_full_noise2
